# Jupiter Notebook for Testing Connections to Personal AI

## Validating Personal.AI API Key

This code demonstrates how to validate the Personal.AI API key by sending a GET request to the `/v1/api-key/validate` endpoint. The environment variables, such as the API key and base URL, are loaded from a `.env` file located in the `config/` directory.

### Key Steps:
1. Load environment variables using `dotenv`.
2. Make a GET request to the API key validation endpoint.
3. Handle the response, including error handling for HTTP exceptions and empty or invalid responses.

Ensure that your `.env` file includes the necessary configurations:

```env
PERSONAL_AI_API_KEY=your_api_key
PERSONAL_AI_BASE_URL=https://api.personal.ai


In [43]:
import os
import requests
import json
from dotenv import load_dotenv
from typing import Dict, Any

class PersonalAIValidator:
    """
    A class to validate the Personal.AI API key.
    """

    def __init__(self, config_path: str = '../config/.env'):
        """
        Initialize the validator by loading environment variables.

        :param config_path: Path to the .env file containing environment variables.
        """
        # * Load environment variables from .env file located in the config folder
        load_dotenv(dotenv_path=config_path)
        
        # * Debug: Print the loaded environment variables (excluding sensitive information)
        print("Loaded Environment Variables:")
        print("PERSONAL_AI_BASE_URL:", os.getenv('PERSONAL_AI_BASE_URL'))
        print("PERSONAL_AI_API_KEY:", "***" + os.getenv('PERSONAL_AI_API_KEY')[-4:])

    def validate_api_key(self) -> Dict[str, Any]:
        """
        Validate the provided API key to ensure it is correct and active.

        :return: A dictionary containing the validation status and user information if successful.
        """
        url = f"{os.getenv('PERSONAL_AI_BASE_URL')}/v1/api-key/validate"
        headers = {
            'Content-Type': 'application/json',
            'x-api-key': os.getenv('PERSONAL_AI_API_KEY')
        }

        try:
            # * Debug: Print the request details (excluding sensitive information)
            print("Request URL:", url)
            print("Request Headers: {'Content-Type': 'application/json', 'x-api-key': '***" + os.getenv('PERSONAL_AI_API_KEY')[-4:] + "'}")

            response = requests.get(url, headers=headers)
            
            # * Debug: Print the full response details
            print(f"Response Status Code: {response.status_code}")
            print(f"Response Headers: {response.headers}")
            print(f"Response Body: {response.text}")

            response.raise_for_status()  # ! Raise an exception for HTTP errors
            
            # * Return the full JSON response for detailed information
            if response.content:
                try:
                    json_response = response.json()
                    print("Parsed JSON Response:", json.dumps(json_response, indent=4))
                except ValueError:
                    # ! Handle JSON decoding error
                    return {"validation": "failed", "error": "Invalid JSON response", "response_body": response.text}
                
                validation_status = json_response.get("validation", "failed")
                
                # ! Check if validation failed despite a 200 status code
                if validation_status != "success":
                    return {"validation": "failed", "error": "Validation failed despite 200 status code, Unable to Return Data for Validation.", "response_body": json_response}
                
                return {
                    "validation": validation_status,
                    "firstName": json_response.get("firstName"),
                    "lastName": json_response.get("lastName"),
                    "email": json_response.get("email"),
                    "response_body": json_response
                }
            else:
                return {"validation": "failed", "error": "Empty response", "response_body": None}
        except requests.exceptions.RequestException as e:
            # ! Print detailed error information
            print(f"An error occurred while validating the API key: {e}")
            print(f"URL: {url}")
            print(f"Headers: {{'Content-Type': 'application/json', 'x-api-key': '***{os.getenv('PERSONAL_AI_API_KEY')[-4:]}'}}")
            return {"validation": "failed", "error": str(e), "response_body": None}

# ! Example usage with proper error handling and detailed output
try:
    validator = PersonalAIValidator()
    validation_response = validator.validate_api_key()
    print("Validation Response:", validation_response)
except Exception as e:
    # ! Print detailed error information if the validation process fails
    print(f"An error occurred during the validation process: {e}")

Loaded Environment Variables:
PERSONAL_AI_BASE_URL: https://api.personal.ai
PERSONAL_AI_API_KEY: ***ec72
Request URL: https://api.personal.ai/v1/api-key/validate
Request Headers: {'Content-Type': 'application/json', 'x-api-key': '***ec72'}
Response Status Code: 200
Response Headers: {'content-type': 'application/json', 'date': 'Mon, 19 Aug 2024 03:15:55 GMT', 'server': 'istio-envoy', 'x-envoy-upstream-service-time': '1', 'transfer-encoding': 'chunked'}
Response Body: {}
Parsed JSON Response: {}
Validation Response: {'validation': 'failed', 'error': 'Validation failed despite 200 status code', 'response_body': {}}


## Sending an External Invite with Personal.AI API

This code demonstrates how to send an email invite to a Personal AI or a Lounge using the Personal.AI API. The environment variables, such as the API key and domain name, are loaded from a `.env` file located in the `config/` directory. The code also includes proper error handling and debug prints to ensure that all steps are executed correctly.

### Key Steps:
1. Load environment variables using `dotenv`.
2. Validate email input.
3. Send the invite and handle potential errors.

Ensure that your `.env` file includes the necessary configurations:

```env
PERSONAL_AI_API_KEY=your_api_key
PERSONAL_AI_DOMAIN_NAME=your_domain_name
PERSONAL_AI_BASE_URL=https://api.personal.ai


In [31]:
import os
import requests
from dotenv import load_dotenv, find_dotenv
from typing import Dict, Any  # * Import Any for type hinting
# * Load environment variables from .env file located in the config folder
dotenv_path = find_dotenv(filename='config/.env', raise_error_if_not_found=False)
if not dotenv_path:
    raise FileNotFoundError("config/.env file not found.")
load_dotenv(dotenv_path=dotenv_path)

def send_external_invite(email: str) -> Dict[str, Any]:
    """
    Send an Email Invite to a Personal AI or a Lounge.

    Args:
        email (str): The email to send the invitation to.

    Returns:
        dict: A dictionary containing the response status and message.
    """
    # * Get domain name from environment variables
    domain_name = os.getenv('PERSONAL_AI_DOMAIN_NAME')
    
    # ! Ensure domain name is set
    if not domain_name:
        raise ValueError("Domain name is not set in the environment variables.")
    
    # * Debug: Print the domain name to ensure it is loaded correctly
    print(f"Domain Name: {domain_name}")
    
    url = f"https://api.personal.ai/v1/invite?email={email}&domain_name={domain_name}"
    headers = {
        'x-api-key': os.getenv('PERSONAL_AI_API_KEY'),
        'Content-Type': 'application/json'
    }

    try:
        response = requests.post(url, headers=headers, data="{}")
        response.raise_for_status()  # ! Raise an exception for HTTP errors

        # * Return the full response for detailed information
        return {
            "status_code": response.status_code,
            "message": response.text
        }
    except requests.exceptions.RequestException as e:
        # ! Print detailed error information
        print(f"An error occurred while sending the external invite: {e}")
        print(f"URL: {url}")
        print(f"Headers: {{'Content-Type': 'application/json', 'x-api-key': '***{os.getenv('PERSONAL_AI_API_KEY')[-4:]}'}}")
        return {
            "status_code": response.status_code if 'response' in locals() else None,
            "error": str(e)
        }

# ! Example usage with proper error handling and detailed output
try:
    # Get email input from user
    email = input("Enter the email to send the invite to: ").strip()
    
    # Validate email input
    if not email or "@" not in email:
        email = "ms@ve7ltx.cc"  # Default email if input is blank or invalid
    
    # * Check if domain name is set before proceeding
    if not os.getenv('PERSONAL_AI_DOMAIN_NAME'):
        raise ValueError("Domain name is not set in the environment variables.")
    
    invite_response = send_external_invite(email=email)
    print("Invite Response:", invite_response)
except FileNotFoundError as fnfe:
    # ! Handle specific FileNotFoundError for .env file not found
    print(f"FileNotFoundError: {fnfe}")
except ValueError as ve:
    # ! Handle specific ValueError for domain name not set
    print(f"ValueError: {ve}")
except Exception as e:
    # ! Print detailed error information if the invite process fails
    print(f"An error occurred during the invite process: {e}")

Domain Name: ms-value
Invite Response: {'status_code': 200, 'message': 'Invite sent to ms@ve7ltx.cc'}


## Sending AI Instructions to Personal.AI

This code demonstrates how to send a message (instruction) to your Personal.AI using the `/v1/instruction?cmd=search` endpoint. The environment variables, including the API key and domain name, are loaded from a `.env` file.

### Key Steps:
1. Load environment variables using `dotenv`.
2. Construct a POST request to send an AI instruction.
3. Handle the response and manage potential errors.

Ensure that your `.env` file includes the necessary configurations:

```env
PERSONAL_AI_API_KEY=your_api_key
PERSONAL_AI_BASE_URL=https://api.personal.ai
PERSONAL_AI_DOMAIN_NAME=your_ai_domain


In [33]:
import os
import requests
from dotenv import load_dotenv

# * Load environment variables from .env file
load_dotenv()

def send_ai_instruction(text: str, context: str = "", username: str = "", session_id: str = "", source_name: str = "", is_stack: bool = False, is_draft: bool = False) -> dict:
    """
    Send an instruction to the Personal.AI API and receive a response.

    Args:
        text (str): Message to send to your AI for a response.
        context (str, optional): Additional context for the AI. Defaults to "".
        username (str, optional): Name of the user sending the request. Defaults to "".
        session_id (str, optional): Session ID to continue the conversation. Defaults to "".
        source_name (str, optional): Name of the source app of the inbound message. Defaults to "".
        is_stack (bool, optional): Flag to add the user message to memory. Defaults to False.
        is_draft (bool, optional): Flag to create a copilot message for the AI. Defaults to False.

    Returns:
        dict: Response from the Personal.AI API.
    """
    url = f"{os.getenv('PERSONAL_AI_BASE_URL')}/v1/instruction?cmd=search"
    payload = {
        "Text": text,
        "Context": context,
        "UserName": username,
        "DomainName": os.getenv('PERSONAL_AI_DOMAIN_NAME'),
        "SessionId": session_id,
        "SourceName": source_name,
        "is_stack": is_stack,
        "is_draft": is_draft
    }
    headers = {
        'x-api-key': os.getenv('PERSONAL_AI_API_KEY'),
        'Content-Type': 'application/json'
    }

    try:
        response = requests.post(url, json=payload, headers=headers)
        response.raise_for_status()  # ! Raise an exception for HTTP errors

        # * Return the full response for detailed information
        return {
            "status_code": response.status_code,
            "message": response.json()
        }
    except requests.exceptions.RequestException as e:
        # ! Print detailed error information
        print(f"An error occurred while sending the AI instruction: {e}")
        print(f"URL: {url}")
        print(f"Headers: {{'Content-Type': 'application/json', 'x-api-key': '***{os.getenv('PERSONAL_AI_API_KEY')[-4:]}'}}")
        return {
            "status_code": response.status_code if 'response' in locals() else None,
            "error": str(e)
        }

# ! Example usage with proper error handling and detailed output
try:
    # Get text input from user
    text = input("Enter the message to send to your AI: ").strip()
    
    # Get context input from user
    context = input("Enter the context for the message (optional): ").strip()
    
    # Validate text input
    if not text:
        text = "This is a test, can you hear me?"  # Default text if input is empty
    
    # * Check if domain name is set before proceeding
    if not os.getenv('PERSONAL_AI_DOMAIN_NAME'):
        raise ValueError("Domain name is not set in the environment variables.")
    
    ai_response = send_ai_instruction(text=text, context=context)
    print("AI Response:", ai_response)
except FileNotFoundError as fnfe:
    # ! Handle specific FileNotFoundError for .env file not found
    print(f"FileNotFoundError: {fnfe}")
except ValueError as ve:
    # ! Handle specific ValueError for invalid input
    print(f"ValueError: {ve}")
except Exception as e:
    # ! Print detailed error information if the AI instruction process fails
    print(f"An error occurred during the AI instruction process: {e}")


AI Response: {'status_code': 200, 'message': {'ai_message': 'The date today is Monday, August 19, 2024.', 'ai_score': 0.0, 'ai_name': 'GOLDBACK', 'ai_picture': 'https://lis-profile-images-prod.s3-us-west-2.amazonaws.com/thumbnail_CVLLVT0V4NG6DQIIKZNA9OND8IA2S1.jpg', 'raw_message': 'The date today is Monday, August 19, 2024.', 'SessionId': '', 'Metadata': {}}}
